In [ ]:
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 3.7 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad
from sympy import Symbol, Derivative

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Data_BlackKoffer_intern/DATA1/40.xls',sheet_name='results',header=None)
df

In [ ]:
def f1(sigma,t):   # when we use this function, t will go from 1 till 300 (not 0-299)
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

We win 2dollars on winning bet and loss $40 on lossing bet

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                # starting time stamp
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:   # sigma = 0 means that we do not have any data yet to calculate sigma
    return 0.0001  # very little probability of hitting target
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),-4)
  result = 1 - 2*(phi)
  if result>=1: 
    result = 0.999   # near certain of hitting the target
  return result    

In [ ]:
def pc():
  return 0.844

In [ ]:
def compute_expValue(t, alpha1, alpha2, alpha3, lamda):
  if t%20==0 or t%20==1: return 0   # t is the first or last data point of a trial set
  Pg = pg(t)
  Pc = pc()
  a = np.exp(-(-math.log(Pg))**alpha3)
  b = lamda*np.exp(-(-math.log(1-Pg))**alpha3)
  c = np.exp(-(-math.log(Pc))**alpha3)
  d = lamda*np.exp(-(-math.log(1-Pc))**alpha3)
  V = Pm1Xt(t-1)*(a*pow(2,alpha1) - b*pow(40,alpha2)) + Pm2Xt(t-1)*(c*pow(2,alpha1) - d*pow(40,alpha2))
  return V # expected value

We bet on trial t+1 if and only if Vt > 0 

In [ ]:

def predict(start, end, alpha1, alpha2, alpha3, lamda):
  total = 0
  plist = []
  expList = []
  for i in range(start,end):
    # i is the time stamp
    Vt = compute_expValue(i, alpha1, alpha2, alpha3, lamda)
    expList.append(Vt)
    if Vt>0:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: 
        total+=2
      else: 
        total-=40
    else: plist.append(0)        
  return (plist,expList,total)       

Now we train the model (for parameters alpha1, alpha2, alpha3 and lamda)

In [ ]:
Y = df[5]

In [ ]:
def errorFunc(prediction, Y):  # cost function
  total = 0
  for i in range(1,160):  # i is timestamp
    #total += (prediction[i]-Y[i])**2
    if (Y[i]==0 and prediction[i]>0) or (Y[i]==1 and prediction[i]<=0): total += prediction[i]**2
  total /= 320
  return total 

In [ ]:
def derivative_cost_function(alpha1,alpha2,alpha3,lamda):  # gradient of cost function (used in gradient descent)
  G_a1 = 0   # total gradient 
  G_a2 = 0
  G_a3 = 0
  G_L = 0
  plist,expList,total = predict(0,160,alpha1,alpha2,alpha3,lamda)
  
  for i in range(3,160):  
    pm1 = Pm1Xt(i-2)
    pm2 = Pm2Xt(i-2)
    Pg = pg(i-1)
    Pc = pc()
    #V = compute_expValue(i-1,a1,a2,a3,L)
    d_a1 = -pm1*(np.exp(-(-math.log(Pg))**alpha3)*2**alpha1*math.log(2)) - pm2*(np.exp(-(-math.log(Pc))**alpha3)*2**alpha1*math.log(2))          #Derivative(V,a1)
    d_a2 = -pm1*lamda*(np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2*math.log(40)) - pm2*lamda*(np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2*math.log(40))
    d_a3 = -pm1*(-(-math.log(Pg))**alpha3*math.log(-math.log(Pg))*np.exp(-(-math.log(Pg))**alpha3)*2**alpha1 + lamda*(-math.log(1-Pg))**alpha3*math.log(-math.log(1-Pg))*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2) - pm2*(-(-math.log(Pc))**alpha3*math.log(-math.log(Pc))*np.exp(-(-math.log(Pc))**alpha3)*2**alpha1 + lamda*(-math.log(1-Pc))**alpha3*math.log(-math.log(1-Pc))*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2)
    d_L = pm1*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2 + pm2*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2
    if (Y[i]==0 and expList[i]>0) or (Y[i]==1 and expList[i]<=0):
      G_a1 += (expList[i])*d_a1
      G_a2 += (expList[i])*d_a2
      G_a3 += (expList[i])*d_a3
      G_L += (expList[i])*d_L
  
  G_a1 /= 140
  G_a2 /= 140
  G_a3 /= 140
  G_L /= 140 
  return (G_a1,G_a2,G_a3,G_L)

In [ ]:
alpha1,alpha2,alpha3,lamda = 1.0,1.0,1.0,1.0

alpha = 0.005  # learning rate
i = 0
while True:
  if i==100: break
  
  if i==0:
    pre_a1,pre_a2,pre_a3,pre_L = alpha1,alpha2,alpha3,lamda
    i+=1
  else:
    G_a1,G_a2,G_a3,G_L = derivative_cost_function(alpha1,alpha2,alpha3,lamda)
    temp1 = alpha1 - alpha*G_a1
    temp2 = alpha2 - alpha*G_a2
    temp3 = alpha3 - alpha*G_a3
    temp4 = lamda - alpha*G_L
    alpha1,alpha2,alpha3,lamda = temp1,temp2,temp3,temp4
    if alpha1>=1: 
      alpha1 = 0.99
    if alpha2>=1:
      alpha2 = 0.99
    if alpha3>=1:
      alpha3 = 0.99
    print("alpha1, alpha2, alpha3, lamda: ",alpha1,alpha2,alpha3,lamda)
    if abs(alpha1-pre_a1)<1e-3 or abs(alpha2-pre_a2)<1e-3 or abs(alpha3-pre_a3)<1e-3 or abs(lamda-pre_L)<1e-3: break
    else: pre_a1,pre_a2,pre_a3,pre_L = alpha1,alpha2,alpha3,lamda
    i+=1

if lamda<=1.01: lamda = 1.01
print("alpha1, alpha2, alpha3, lamda: ",alpha1,alpha2,alpha3,lamda)

Now we obtain the results

In [ ]:
#plist,total = predict(160,300,alpha1,alpha2,alpha3,lamda)
plist,expList,total = predict(160,300,alpha1,alpha2,alpha3,lamda)
print(len(plist))
print(total)

In [ ]:
print(plist)

In [ ]:
Y = df[5]
total = 0
for i in range(0,140):
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model is: ",(total/140)*100)  